In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [11]
commitnumber = de5d43e
molecule = {'co2': 0.0004}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [3, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]
  wgt = [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.132474                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.266759
0.000750      2.0  0.010364                  2.0  0.088075
0.001052      3.0  0.006665                  3.0  0.045955
0.001476      4.0  0.001568                  4.0  0.009683
0.002070      5.0 -0.003415                  5.0 -0.016890
0.002904      6.0 -0.007743                  6.0 -0.031918
0.004074      7.0 -0.011661                  7.0 -0.037643
0.005714      8.0 -0.015487                  8.0 -0.039549
0.008015      9.0 -0.019382                  9.0 -0.039600
0.011243     10.0 -0.023514                 10.0 -0.040245
0.015771     11.0 -0.027995                 11.0 -0.041200
0.022122     12.0 -0.032999                 12.0 -0.043683
0.031031     13.0 -0.039068                 13.0 -0.048002
0.043528     14.0 -0.049244                 14.0 -0.057882
0.061057     15.0 -0.061178                 15.0 -0.068914
0.085645     16.0 -0.065256                 16.0 -0.067396
0.120136     17.0 -0.061071                 17.0 -0.052230
0.168516     18.0 -0.055987                 18.0 -0.034503
0.236378     19.0 -0.045017                 19.0 -0.016058
0.331549     20.0 -0.022706                 20.0  0.001781
0.465100     21.0  0.020035                 21.0  0.024721
0.652400     22.0  0.116627                 22.0  0.095150
0.915100     23.0  0.267705                 23.0  0.228004
1.283650     24.0  0.266036                 24.0  0.237629
1.800600     25.0  0.149019                 25.0  0.163440
2.525700     26.0  0.080983                 26.0  0.113222
3.542800     27.0  0.044815                 27.0  0.069969
4.969550     28.0  0.023997                 28.0  0.031058
6.970850     29.0  0.012753                 29.0  0.007448
9.778100     30.0  0.006386                 30.0  0.002980
13.715850    31.0  0.001971                 31.0  0.004983
19.239350    32.0 -0.000880                 32.0  0.003142
26.987250    33.0 -0.001438                 33.0  0.000496
37.855300    34.0 -0.001031                 34.0 -0.000944
53.100050    35.0 -0.001162                 35.0 -0.001469
73.887500    36.0 -0.001621                 36.0 -0.001767
97.662500    37.0 -0.001857                 37.0 -0.001906
121.437500   38.0 -0.001955                 38.0 -0.001950
145.212500   39.0 -0.002453                 39.0 -0.002497
168.987500   40.0 -0.003266                 40.0 -0.003423
192.762500   41.0 -0.003714                 41.0 -0.003904
216.537500   42.0 -0.003539                 42.0 -0.003750
240.312500   43.0 -0.003489                 43.0 -0.003802
264.087500   44.0 -0.003526                 44.0 -0.003933
287.862500   45.0 -0.003551                 45.0 -0.004002
311.637500   46.0 -0.003547                 46.0 -0.003992
335.412500   47.0 -0.003508                 47.0 -0.003928
359.187500   48.0 -0.003430                 48.0 -0.003797
382.962500   49.0 -0.003305                 49.0 -0.003579
406.737500   50.0 -0.003131                 50.0 -0.003301
430.512500   51.0 -0.002920                 51.0 -0.002953
454.287500   52.0 -0.002697                 52.0 -0.002615
478.062500   53.0 -0.002482                 53.0 -0.002247
501.837500   54.0 -0.002273                 54.0 -0.001924
525.612500   55.0 -0.002051                 55.0 -0.001585
549.387500   56.0 -0.001802                 56.0 -0.001226
573.162500   57.0 -0.001522                 57.0 -0.000864
596.937500   58.0 -0.001208                 58.0 -0.000490
620.712500   59.0 -0.000860                 59.0 -0.000111
644.487500   60.0 -0.000478                 60.0  0.000330
668.262500   61.0 -0.000057                 61.0  0.000789
692.037500   62.0  0.000407                 62.0  0.001274
715.812500   63.0  0.000926                 63.0  0.001799
739.587500   64.0  0.001518                 64.0  0.002498
76

# Fluxes by Layer

CRD                        CLIRAD                    
                        flug      fldg     fnetg      flug      fldg     fnetg
pressure     level                                                            
0.000000e+00 1     -5.482672  0.000000 -5.482672       NaN       NaN       NaN
1.000000e-08 1           NaN       NaN       NaN -5.396187  0.000002 -5.396185
6.244000e-04 2     -5.482677  0.000014 -5.482662 -5.396194  0.000029 -5.396165
8.759000e-04 3     -5.482679  0.000017 -5.482662 -5.396197  0.000034 -5.396163
1.228600e-03 4     -5.482681  0.000019 -5.482662 -5.396201  0.000040 -5.396161
1.723400e-03 5     -5.482685  0.000023 -5.482662 -5.396206  0.000046 -5.396160
2.417400e-03 6     -5.482690  0.000028 -5.482662 -5.396213  0.000051 -5.396162
3.390900e-03 7     -5.482696  0.000033 -5.482663 -5.396222  0.000057 -5.396165
4.756500e-03 8     -5.482705  0.000040 -5.482665 -5.396235  0.000063 -5.396171
6.672000e-03 9     -5.482716  0.000048 -5.482668 -5.396251  0.000071 -5.396180
9.358900e-03 10    -5.482733  0.000058 -5.482674 -5.396275  0.000082 -5.396193
1.312780e-02 11    -5.482755  0.000070 -5.482685 -5.396307  0.000096 -5.396211
1.841450e-02 12    -5.482787  0.000085 -5.482703 -5.396352  0.000115 -5.396237
2.583020e-02 13    -5.482834  0.000102 -5.482731 -5.396417  0.000142 -5.396275
3.623230e-02 14    -5.482902  0.000122 -5.482780 -5.396512  0.000178 -5.396334
5.082340e-02 15    -5.483011  0.000146 -5.482865 -5.396661  0.000226 -5.396434
7.129060e-02 16    -5.483198  0.000185 -5.483013 -5.396903  0.000301 -5.396602
1.000000e-01 17    -5.483512  0.000277 -5.483235 -5.397283  0.000452 -5.396831
1.402710e-01 18    -5.484008  0.000482 -5.483526 -5.397848  0.000768 -5.397080
1.967600e-01 19    -5.484781  0.000880 -5.483901 -5.398676  0.001365 -5.397311
2.759970e-01 20    -5.485958  0.001634 -5.484324 -5.399873  0.002411 -5.397462
3.871000e-01 21    -5.487680  0.003057 -5.484622 -5.401557  0.004119 -5.397438
5.431000e-01 22    -5.489987  0.005734 -5.484252 -5.403787  0.006806 -5.396981
7.617000e-01 23    -5.491951  0.010719 -5.481232 -5.405668  0.011151 -5.394517
1.068500e+00 24    -5.490177  0.018674 -5.471503 -5.403826  0.017596 -5.386230
1.498800e+00 25    -5.483984  0.026041 -5.457943 -5.397973  0.023857 -5.374116
2.102400e+00 26    -5.476854  0.029567 -5.447288 -5.391205  0.028776 -5.362429
2.949000e+00 27    -5.470008  0.030842 -5.439166 -5.384489  0.033416 -5.351073
4.136600e+00 28    -5.463736  0.030874 -5.432862 -5.377996  0.036767 -5.341229
5.802500e+00 29    -5.458185  0.030059 -5.428126 -5.371912  0.036813 -5.335100
8.139200e+00 30    -5.453277  0.028681 -5.424596 -5.366179  0.033141 -5.333038
1.141700e+01 31    -5.448881  0.026764 -5.422117 -5.360736  0.028856 -5.331881
1.601470e+01 32    -5.445177  0.024134 -5.421043 -5.355922  0.026755 -5.329166
2.246400e+01 33    -5.442687  0.020972 -5.421715 -5.352428  0.025662 -5.326766
3.151050e+01 34    -5.441471  0.018215 -5.423256 -5.350240  0.024005 -5.326234
4.420010e+01 35    -5.441209  0.016402 -5.424807 -5.348918  0.021264 -5.327654
6.200000e+01 36    -5.442347  0.015091 -5.427256 -5.348977  0.018224 -5.330752
8.577500e+01 37    -5.445970  0.014148 -5.431822 -5.351955  0.016225 -5.335730
1.095500e+02 38    -5.451056  0.014005 -5.437051 -5.356879  0.015781 -5.341098
1.333250e+02 39    -5.457069  0.014511 -5.442558 -5.362918  0.016326 -5.346592
1.571000e+02 40    -5.464547  0.015081 -5.449465 -5.370613  0.016989 -5.353625
1.808750e+02 41    -5.474803  0.016141 -5.458662 -5.381575  0.018310 -5.363265
2.046500e+02 42    -5.488307  0.019187 -5.469121 -5.396267  0.022007 -5.374259
2.284250e+02 43    -5.504178  0.025090 -5.479088 -5.413583  0.028760 -5.384823
2.522000e+02 44    -5.522097  0.033184 -5.488913 -5.433097  0.037566 -5.395531
2.759750e+02 45    -5.542040  0.043197 -5.498843 -5.454798  0.048189 -5.406608
2.997500e+02 46    -5.563993  0.055150 -5.508843 -5.478661  0.060781 -5.417880
3.235250e+02 47    -5.587932  0.069099 -5.518833 -5.504599  0.075475 -5.429

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')